In [7]:
from OpenSSL import crypto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import time
from difflib import SequenceMatcher

In [8]:
def normalise_counter(ctr):
    total = sum(ctr.values())
    for key in ctr.keys():
        ctr[key] /=total
    return ctr

In [9]:
def get_existing_certs(distinct_urls,folder):
    successes = []
    data_path = folder+'/'
    for url in distinct_urls:
        try:
            f = open(data_path+url)    
            if f.readline() != '':
                successes.append(url)
        except:
            pass
    return successes

In [10]:
phish_df = pd.read_csv('final_certificate_datasets/phish_dataset_urls.csv')
benign_df = pd.read_csv('final_certificate_datasets/benign_certs.csv')

phish_total = len(phish_df)
benign_total = len(benign_df)

In [11]:
phish_df

,Unnamed: 0,url,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected,target
0,0,http://sillyabba.com/abut/Adobe1/Adobe1/login....,*.hostgator.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200903000000Z',b'20210903235959Z',0,1.620480e+09,Other
1,1,https://www.cybersolution.eu/,*.aruba.it,Aruba S.p.A.,IT,NaN,NaN,NaN,Actalis Organization Validated Server CA G3,Actalis S.p.A.,0.0,b'20200723123537Z',b'20220723123537Z',0,1.620480e+09,Other
2,2,https://dopeydog.co.nz/wp-includes/index.html?...,Plesk,Odin,US,NaN,NaN,NaN,Plesk,Odin,1.0,b'20160428040009Z',b'20170428040009Z',10,1.620480e+09,Other
3,3,http://unrecognisedrequestedpayee.com/lloyds/L...,raa.namecheap.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20201111000000Z',b'20211129235959Z',0,1.620480e+09,Other
4,4,https://paypal.me.holdpaystatic.shop/connexion,*.web-hosting.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200507000000Z',b'20220405235959Z',0,1.620480e+09,PayPal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13972,561,http://u11073366bg.ha004.t.justns.ru/société_g...,*.justhost.ru,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20201108000000Z',b'20211108235959Z',21,1.620495e+09,Other
13973,564,http://vivianegibert.com/Adpadpsecurity/adp,vivianegibert.com,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210508153722Z',b'20210806153722Z',0,1.620494e+09,Other
13974,565,http://fbtimeline-1jhlo4ji.villalarnia.org/con...,villalarnia.org,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210326064357Z',b'20210624064357Z',0,1.620495e+09,Other
13975,566,https://fbtimeline-1jhlo4ji.villalarnia.org/co...,villalarnia.org,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210326064357Z',b'20210624064357Z',0,1.620495e+09,Other


In [12]:
distinct_df = phish_df.drop_duplicates(subset=['CN'])
distinct_df

,Unnamed: 0,url,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected,target
0,0,http://sillyabba.com/abut/Adobe1/Adobe1/login....,*.hostgator.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200903000000Z',b'20210903235959Z',0,1.620480e+09,Other
1,1,https://www.cybersolution.eu/,*.aruba.it,Aruba S.p.A.,IT,NaN,NaN,NaN,Actalis Organization Validated Server CA G3,Actalis S.p.A.,0.0,b'20200723123537Z',b'20220723123537Z',0,1.620480e+09,Other
2,2,https://dopeydog.co.nz/wp-includes/index.html?...,Plesk,Odin,US,NaN,NaN,NaN,Plesk,Odin,1.0,b'20160428040009Z',b'20170428040009Z',10,1.620480e+09,Other
3,3,http://unrecognisedrequestedpayee.com/lloyds/L...,raa.namecheap.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20201111000000Z',b'20211129235959Z',0,1.620480e+09,Other
4,4,https://paypal.me.holdpaystatic.shop/connexion,*.web-hosting.com,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20200507000000Z',b'20220405235959Z',0,1.620480e+09,PayPal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13961,545,https://ahjijiautomobile.de/suchen.mobile.de.f...,ahjijiautomobile.de,NaN,NaN,NaN,NaN,NaN,GeoTrust TLS DV RSA Mixed SHA256 2020 CA-1,DigiCert Inc,0.0,b'20210506000000Z',b'20220505235959Z',0,1.620495e+09,"eBay, Inc."
13962,546,http://grup-handai-wa18.duckdns.org/,grup-handai-wa18.duckdns.org,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",0.0,b'20210508000000Z',b'20210806235959Z',0,1.620495e+09,Other
13963,551,https://jcb-jcb.cc/,jcb-jcb.cc,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210506161457Z',b'20210804161457Z',0,1.620495e+09,Other
13973,564,http://vivianegibert.com/Adpadpsecurity/adp,vivianegibert.com,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210508153722Z',b'20210806153722Z',0,1.620494e+09,Other


In [13]:
ctr = Counter(distinct_df['target'])
ctr.most_common(11)

[('Other', 3132),
 ('eBay, Inc.', 47),
 ('Microsoft', 46),
 ('Internal Revenue Service', 46),
 ('PayPal', 44),
 ('Facebook', 29),
 ('Capitec Bank', 24),
 ('Amazon.com', 23),
 ('ABSA Bank', 22),
 ('Vodafone', 16),
 ('ING Direct', 14)]

In [14]:
ctr2 = Counter(phish_df['target'])
ctr2.most_common(11)

[('Other', 12172),
 ('Facebook', 261),
 ('eBay, Inc.', 213),
 ('PayPal', 180),
 ('RuneScape', 164),
 ('Microsoft', 134),
 ('Internal Revenue Service', 99),
 ('Amazon.com', 70),
 ('PKO Polish Bank', 51),
 ('Vodafone', 46),
 ('Orange', 41)]

In [15]:
targets = []
for e in ctr.most_common(11)[1:]:
    targets.append(e[0])
targets

['eBay, Inc.',
 'Microsoft',
 'Internal Revenue Service',
 'PayPal',
 'Facebook',
 'Capitec Bank',
 'Amazon.com',
 'ABSA Bank',
 'Vodafone',
 'ING Direct']

In [72]:
CNs = []
Os = []
ICNs = []
IOs = []

In [73]:
CNs.append('www.ebay.com')
Os.append('eBay, Inc.')
ICNs.append('DigiCert TLS RSA SHA256 2020 CA1')
IOs.append("DigiCert Inc")

In [74]:
CNs.append('www.microsoft.com')
Os.append('Microsoft Corporation')
ICNs.append('Microsoft RSA TLS CA 01')
IOs.append("Microsoft Corporation")

In [75]:
CNs.append('www.irs.gov')
Os.append('Internal Revenue Service')
ICNs.append('Entrust Certification Authority - L1K')
IOs.append("Entrust, Inc.")

In [76]:
CNs.append('www.paypal.com')
Os.append('PayPal, Inc.')
ICNs.append('DigiCert SHA2 Extended Validation Server CA')
IOs.append("DigiCert Inc")

In [77]:
CNs.append('*.facebook.com')
Os.append('Facebook, Inc.')
ICNs.append('DigiCert SHA2 High Assurance Server CA')
IOs.append("DigiCert Inc")

In [78]:
CNs.append('www.capitecbank.co.za')
Os.append('Capitec Bank Limited')
ICNs.append('Entrust Certification Authority - L1M')
IOs.append("Entrust, Inc.")

In [79]:
CNs.append('www.amazon.co.uk')
Os.append('Amazon.com, Inc.')
ICNs.append('DigiCert Global CA G2')
IOs.append("DigiCert Inc")

In [80]:
CNs.append('www.absa.africa')
Os.append('Absa Group Limited')
ICNs.append('DigiCert SHA2 Extended Validation Server CA')
IOs.append("DigiCert Inc")

In [81]:
CNs.append('www.vodafone.co.uk')
Os.append('Vodafone Group Services Limited')
ICNs.append('DigiCert SHA2 Secure Server CA')
IOs.append("DigiCert Inc")

In [82]:
CNs.append('www.ing.com')
Os.append('ING Groep N.V.')
ICNs.append('Entrust Certification Authority - L1M')
IOs.append("Entrust, Inc.")

In [85]:
target_df = pd.DataFrame()
target_df['target'] = targets
target_df['CN'] = CNs
target_df['O'] = Os
target_df['ICN'] = ICNs
target_df['IO'] = IOs
target_df

,target,CN,O,ICN,IO
0,"eBay, Inc.",www.ebay.com,"eBay, Inc.",DigiCert TLS RSA SHA256 2020 CA1,DigiCert Inc
1,Microsoft,www.microsoft.com,Microsoft Corporation,Microsoft RSA TLS CA 01,Microsoft Corporation
2,Internal Revenue Service,www.irs.gov,Internal Revenue Service,Entrust Certification Authority - L1K,"Entrust, Inc."
3,PayPal,www.paypal.com,"PayPal, Inc.",DigiCert SHA2 Extended Validation Server CA,DigiCert Inc
4,Facebook,*.facebook.com,"Facebook, Inc.",DigiCert SHA2 High Assurance Server CA,DigiCert Inc
5,Capitec Bank,www.capitecbank.co.za,Capitec Bank Limited,Entrust Certification Authority - L1M,"Entrust, Inc."
6,Amazon.com,www.amazon.co.uk,"Amazon.com, Inc.",DigiCert Global CA G2,DigiCert Inc
7,ABSA Bank,www.absa.africa,Absa Group Limited,DigiCert SHA2 Extended Validation Server CA,DigiCert Inc
8,Vodafone,www.vodafone.co.uk,Vodafone Group Services Limited,DigiCert SHA2 Secure Server CA,DigiCert Inc
9,ING Direct,www.ing.com,ING Groep N.V.,Entrust Certification Authority - L1M,"Entrust, Inc."


In [31]:
df2 = distinct_df.loc[distinct_df['target'] !='Other']
Counter(df2['target']).most_common(10)

[('eBay, Inc.', 47),
 ('Microsoft', 46),
 ('Internal Revenue Service', 46),
 ('PayPal', 44),
 ('Facebook', 29),
 ('Capitec Bank', 24),
 ('Amazon.com', 23),
 ('ABSA Bank', 22),
 ('Vodafone', 16),
 ('ING Direct', 14)]

In [32]:
target_dict = dict()
for t in targets:
    subset = df2.loc[df2['target']==t]
    target_dict[t] = subset
    

In [33]:
target_dict['Microsoft']

,Unnamed: 0,url,CN,O,C,businessCategory,serialNumber,juristiction,issuer_CN,issuer_O,expired,notBefore,notAfter,verCode,time_collected,target
184,184,https://webb.cabanova.com/,*.cabanova.com,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210315221209Z',b'20210613221209Z',0,1.620480e+09,Microsoft
653,653,https://www.smartblackout.com/crossventure/ofc...,abogadomedellin.com.co,NaN,NaN,NaN,NaN,NaN,"cPanel, Inc. Certification Authority","cPanel, Inc.",1.0,b'20200509000000Z',b'20200807235959Z',10,1.620480e+09,Microsoft
1050,1050,https://outlooks-initial-project-24b704.webflo...,t.ssl.fastly.net,"Fastly, Inc",US,NaN,NaN,NaN,GlobalSign CloudSSL CA - SHA256 - G3,GlobalSign nv-sa,0.0,b'20210331211042Z',b'20220401211042Z',0,1.620480e+09,Microsoft
1054,1054,https://protect.theresortweddings.com/,build-our-future.com,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210405070804Z',b'20210704070804Z',0,1.620480e+09,Microsoft
1252,1252,https://spikenow.com/collab/?id=BhyAAYPay0MIZb...,spikenow.com,NaN,NaN,NaN,NaN,NaN,Amazon,Amazon,0.0,b'20200621000000Z',b'20210721120000Z',0,1.620480e+09,Microsoft
1665,1665,http://dh.nortenet.com.ve/#66468406f7265676f6e...,intervenweb.com,NaN,NaN,NaN,NaN,NaN,R3,Let's Encrypt,0.0,b'20210331012631Z',b'20210629012631Z',0,1.620480e+09,Microsoft
2786,2786,https://phillipmill176545.clickfunnels.com/opt...,ssl566619.cloudflaressl.com,NaN,NaN,NaN,NaN,NaN,COMODO ECC Domain Validation Secure Server CA 2,COMODO CA Limited,0.0,b'20210406000000Z',b'20211013235959Z',0,1.620480e+09,Microsoft
3236,3236,https://ninetydegree.com/jh/office/,www.ninetydegree.com,NINETY DEGREE STONE PVT. LTD.,IN,NaN,NaN,NaN,GeoTrust RSA CA 2018,DigiCert Inc,0.0,b'20200912000000Z',b'20211012120000Z',0,1.620480e+09,Microsoft
3287,3287,https://verificationmessage.blob.core.windows....,*.blob.core.windows.net,NaN,NaN,NaN,NaN,NaN,Microsoft RSA TLS CA 01,Microsoft Corporation,0.0,b'20210216210920Z',b'20220216210920Z',0,1.620480e+09,Microsoft
3819,3819,https://sharelink.sn.am/lYPBgpwGauq,*.sn.am,NaN,NaN,NaN,NaN,NaN,Sectigo RSA Domain Validation Secure Server CA,Sectigo Limited,0.0,b'20201202000000Z',b'20211202235959Z',0,1.620480e+09,Microsoft


In [91]:
similar_Os = np.zeros(len(targets),dtype=int)
for i,t in enumerate(targets):
    s1 = target_df.loc[i]['O']
    for j,s2 in enumerate(target_dict[t].loc[target_dict[t]['O'].notna()]['O']):
        r = SequenceMatcher(None, s1, s2).ratio()
        if r >= 0.6:#0.3 creates wrong matches which ahd to be removed
            print("s1: {} | s2: {} | r: {}".format(s1,s2,r))
            print(target_dict[t].loc[target_dict[t]['O']==s2]['url'])
            print(t)
            similar_Os[i] +=1

s1: Microsoft Corporation | s2: Microsoft Corporation | r: 1.0
4808    https://asiadiscoversolutions.azureedge.net/#
Name: url, dtype: object
Microsoft


In [92]:
similar_ICNs = np.zeros(len(targets),dtype=int)
for i,t in enumerate(targets):
    s1 = target_df.loc[i]['ICN']
    for j,s2 in enumerate(target_dict[t].loc[target_dict[t]['issuer_CN'].notna()]['issuer_CN']):
        r = SequenceMatcher(None, s1, s2).ratio()
        if r >= 0.8:#0.3 creates wrong matches which ahd to be removed
            print("s1: {} | s2: {} | r: {}".format(s1,s2,r))
            print(target_dict[t].loc[target_dict[t]['issuer_CN']==s2]['url'])
            similar_ICNs[i] +=1

s1: Microsoft RSA TLS CA 01 | s2: Microsoft RSA TLS CA 01 | r: 1.0
3287    https://verificationmessage.blob.core.windows....
Name: url, dtype: object


In [93]:
similar_IOs = np.zeros(len(targets),dtype=int)
for i,t in enumerate(targets):
    s1 = target_df.loc[i]['IO']
    for j,s2 in enumerate(target_dict[t].loc[target_dict[t]['issuer_O'].notna()]['issuer_O']):
        r = SequenceMatcher(None, s1, s2).ratio()
        if r >= 0.8:#0.3 creates wrong matches which ahd to be removed
            print("s1: {} | s2: {} | r: {}".format(s1,s2,r))
            #print(target_dict[t].loc[target_dict[t]['issuer_O']==s2]['url'])
            similar_IOs[i] +=1

s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: Microsoft Corporation | s2: Microsoft Corporation | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0
s1: DigiCert Inc | s2: DigiCert Inc | r: 1.0


In [94]:
target_counts = [c[1] for c in ctr.most_common(11)[1:]]

In [96]:
final_df = pd.DataFrame()
final_df['Target Organisation'] = target_df['target']
final_df['Common Name'] = target_df['CN']
final_df['Total Phishing Domains Found'] = target_counts
final_df['Same/Similar Organisation'] = similar_Os
final_df['Identical Issuer CN'] = similar_ICNs
final_df['Same Issuer Organisation'] = similar_IOs
final_df

,Target Organisation,Common Name,Total Phishing Domains Found,Same/Similar Organisation,Identical Issuer CN,Same Issuer Organisation
0,"eBay, Inc.",www.ebay.com,47,0,0,8
1,Microsoft,www.microsoft.com,46,1,1,1
2,Internal Revenue Service,www.irs.gov,46,0,0,0
3,PayPal,www.paypal.com,44,0,0,2
4,Facebook,*.facebook.com,29,0,0,2
5,Capitec Bank,www.capitecbank.co.za,24,0,0,0
6,Amazon.com,www.amazon.co.uk,23,0,0,0
7,ABSA Bank,www.absa.africa,22,0,0,1
8,Vodafone,www.vodafone.co.uk,16,0,0,0
9,ING Direct,www.ing.com,14,0,0,0


In [100]:
target_df.loc[0]['CN']

'www.ebay.com'

In [1]:
for i,t in enumerate(targets):
    s1 = target_df.loc[i]['ICN']
    for s2 in target_dict[t].loc[target_dict[t]['issuer_CN'].notna()]['issuer_CN']:
        r = SequenceMatcher(None, s1, s2).ratio()
        if r >= 0.3:
            print("t:{} | s1: {} | s2: {} | r: {}".format(t,s1,s2,r))
            

NameError: name 'targets' is not defined

In [114]:
for i,t in enumerate(targets):
    s1 = t
    for s2 in target_dict[t].loc[target_dict[t]['url'].notna()]['url']:
        r = SequenceMatcher(None, s1, s2).ratio()
        if r >= 0.25:
            print("s1: {} | s2: {} | r: {}".format(s1,s2,r))
            

s1: eBay, Inc. | s2: https://energygain.co.uk/r/ | r: 0.2702702702702703
s1: eBay, Inc. | s2: http://prideloans.co.uk/mbl/ | r: 0.2631578947368421
s1: Microsoft | s2: https://admin.baragor.se/fx3/190.html | r: 0.2608695652173913
s1: Microsoft | s2: http://turboflightpros.com | r: 0.2857142857142857
s1: Microsoft | s2: https://www.microsoftpasswordreactivation.com/ | r: 0.2909090909090909
s1: Microsoft | s2: https://microsoft-53.jimdosite.com | r: 0.37209302325581395
s1: Internal Revenue Service | s2: http://control-infotech.com/irs.gov.authentication.service.request?access | r: 0.26804123711340205
s1: Internal Revenue Service | s2: https://medicaldevicetrainings.com/stipay.html#4 | r: 0.25
s1: Capitec Bank | s2: http://ostad313.ir/ndcs.pni/ecca.php | r: 0.25
s1: Capitec Bank | s2: https://lano.my/ndcs.pni/ecca.php | r: 0.26666666666666666
s1: Amazon.com | s2: https://www.amazccon.co-jp.cc/ | r: 0.4
s1: Amazon.com | s2: https://my2ktop.company.site | r: 0.3157894736842105
s1: Amazon.com